<div id="colab_button">
  <h1>Uploading models with BlindAI</h1>
  <a target="_blank" href="LINK"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

_____________________________________________________________________

One of the first steps for AI engineers using BlindAI to serve their models with privacy guarantees is to upload their models to their BlindAI server instance. This can be done using our easy-to-use Python API, but first, your models need to be converted to the Open Neural Network Exchange Format (ONNX) format.

ONNX is a standard enabling framework interoperability, allowing you to easily move models between different machine learning libraries.

In this tutorial, we will show you how to take models from three of the most popular ML libraries, `PyTorch`, `TensorFlow` and `HuggingFace`, convert them to ONNX format, and upload them to BlindAI.

So, let's dive in.

## Pre Requisites
_______________________________________

### Installing required dependencies

Unless you're are running this notebook on Google Colab, you'll need to have [`Python`](https://www.python.org/downloads/) (3.8 or greater) and [`pip`](https://pypi.org/project/pip/) installed to run this notebook.

Then, you'll need to install the BlindAI-preview package.

In [ ]:
# install blindai-preview package
!pip install blindai-preview

We will also need to install some additional dependencies for this notebook:

- [`torch`](https://pytorch.org/): to demo ONNX conversion for PyTorch models
- [`tensorflow`](https://www.tensorflow.org/): to demo ONNX conversion for TensorFlow models
- [`transformers`](https://huggingface.co/): to demo ONNX conversion for HuggingFace models

In [ ]:
# install all other required packages
!pip install torch tensorflow transformers

### Launch the BlindAI server

Before we can upload any models, we will first need to launch an instance of BlindAI's server. 

For the purposes of this tutorial, we will use the `blindai_preview.testing` server which has been designed for testing purposes only.

In [ ]:
# import testing submodule
import blindai_preview.testing

# start the server
srv = blindai_preview.testing.start_mock_server()

>Note that the blindai-preview testing module launches the server in simulation mode. As a result, it doesn't provide hardware protection and **must not be used in production**. We created this option to enable users to quickly and easily test BlindAI without needing to run the server on a machine with Intel SGX hardware. To learn more about how to run the server with hardware protections on, see [our documentation](https://blindai-preview.mithrilsecurity.io/en/latest/).

### PyTorch: Download model

We will use the pretrained `resnet18` image-classifying neural network as our example model for this section. 

The first step is to download the model from `PyTorch Hub`. We do this using the `hub` module's `load` method. 

The first argument we provide specifies the Github repo and directory where the model can be installed from. We then specify the name of the model to be downloaded and set the `pretrained` option to `True`.

In [ ]:
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

It's important to call `model.eval()` or `model.train(False)` before exporting the model to ensure the model is set to inference mode. 

In [ ]:
model.eval()

### PyTorch: ONNX conversion

We will use the `torch.onnx.export()` function to convert our model to ONNX fomat.

The `export` method will execute the model, recording a trace of what operators are used to compute the outputs. To do this, we need to supply it with some dummy inputs in the shape the model would expect. 

For resnet18, the dummy input should be in the following format: `batch size`, `channel width`, `image size`, `image size`.

The values we provide for each dimension will be fixed in the exported ONNX graph, meaning all future input must match this shape. There is a way around this however, by specifying axes as a dynamic axes.

Let's create our dummy input now.

In [ ]:
# create dummy inputs for resnet18 model
dummy_inputs = torch.zeros(1,3,224,224)

The channel width should be 3, representing the RGB values of our image, and the image size should be 224-by-224. 

We will set the `batch_size` to 1, but then specify the first dimension as dynamic with the `dynamic_axes` option.

Since the values used for our dummy input are not important- here, we will fill our dummy input with zeros.

Now that we have created our dummy inputs, we are ready to call the `onnx.export()` method. We pass the method our PyTorch model, the dummy inputs, the name we want to give our ONNX file and finally any dynamic axes for input and output values.

In [ ]:
torch.onnx.export(model, dummy_inputs, 
                "resnet18.onnx", 
                dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})

> For information about more export options, see the `torch.onnx.export` [documentation](https://pytorch.org/docs/stable/onnx.html#torch.onnx.export).

We have now successfully converted our PyTorch model ready to be uploading to the server.

### Uploading model

Before we can upload the model, we need to connect to the server using BlindAI's `connect()` function.

Since we are using a server in `simulation` mode, we need to set the `simulation_mode` parameter to `True` on the client side also. This is needed because the client will refuse to connect to an unsecure server otherwise.  

>Note, we also set the `hazmat_http_on_unattested_port` option to `True`. By default, the `blindai_preview` package requires a HTTPS connection for communications between the client and server on the unattested port 9923. But for testing purposes we opt out of this requirement and connect without a secure connection. This should not be done in production, please refer to our documentation to set up a production server.

In [1]:
import blindai_preview 
# import blindai_preview

# AI company connects
client_1 = blindai_preview.connect(addr="localhost", simulation_mode=True, hazmat_http_on_untrusted_port=True)

> For the purposes of this demo, we are running the server on localhost using the default ports, but note that the host and ports are modifiable parameters in the `connect()` function.

We can now go ahead and upload the model using the client `upload_model()` method, specifying the ONNX model's file name via the `model` parameter.

We can store and print out our `model_id`, which is used to identify the model when running or deleting the model.

In [ ]:
# AI company uploads model to server
response = client_1.upload_model(model="./resnet18.onnx")
MODEL_ID = response.model_id
print(MODEL_ID)

The PyTorch model has now been successfully uploaded to the BlindAI server and is ready to be consumed by users.

### TensorFlow: Download model and ONNX conversion

### HuggingFace: Download model and ONNX convert

The model is now uploaded and we have a `model_id` we can share with our client. 

We, PixelHealth, can now close our connection to the server using the `close()` method.

In [ ]:
# disconnect from sever
client_1.close()

## Conclusions
____________________________________

This is the end of our introduction to BlindAI! 

We have seen how to:

* **Connect** and **disconnect** to the BlindAI server.
* **Upload**, **run** and **delete** models.
* **Prepare** image data for the Covid-Net model.

Please check out the rest of our [BlindAI documentation](https://blindai-preview.mithrilsecurity.io/en/latest/) to see more examples of how you can use BlindAI to deploy AI models without compromising the safety of user data or models.